In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='1.12.0',image_scope='inference',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker pull 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310

In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 049694889102.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker images

In [25]:
!docker build -t 049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo -f Dockerfile.train .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (9/9) FINISHED                                 docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 303B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [1/4] FROM 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-tr  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 64B                                           0.0s
 => CACHED [2/4] ADD ./requirements.txt /opt/ml/code/requirements.txt      0.0s
 => CACHED [3/4] RUN pip install -r /opt/ml/code/requirements.txt          0.0s
 => CACHED [4/4] ADD ./main.py /opt/ml/c

In [ ]:
!docker images

In [26]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo',
    role=role,
    instance_count=1,
    instance_type='local',
    hyperparameters=hyperparameters,
    output_path='file://{}/data/output'.format(os.getcwd())
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('file://{}/data/input/cancer_data.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-11-05-15-13-48-543
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.local.image:'Docker Compose' is not installed. Proceeding to check for 'docker-compose' CLI.
INFO:sagemaker.local.image:'Docker Compose' found using Docker Compose CLI.
INFO:sagemaker.local.local_session:Starting training job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials fou

##### ESTIMATOR FIT STARTED
 Container e6h503e4ro-algo-1-4f9lx  Creating
 Container e6h503e4ro-algo-1-4f9lx  Created
Attaching to e6h503e4ro-algo-1-4f9lx
e6h503e4ro-algo-1-4f9lx  | 2024-11-05 15:13:49.539016: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
e6h503e4ro-algo-1-4f9lx  | 2024-11-05 15:13:49.542563: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
e6h503e4ro-algo-1-4f9lx  | 2024-11-05 15:13:49.554427: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
e6h503e4ro-algo-1-4f9lx  | WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
e6h503e4ro-algo-1-4f9lx  | E0000 00:00:1730819629.573824       1 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register facto

INFO:sagemaker.local.image:===== Job Complete =====


e6h503e4ro-algo-1-4f9lx exited with code 0
Aborting on container exit...
 Container e6h503e4ro-algo-1-4f9lx  Stopping
 Container e6h503e4ro-algo-1-4f9lx  Stopped
##### ESTIMATOR FIT COMPLETED


In [ ]:
!tar -xzf data/output/output.tar.gz

In [27]:
!tar -xzf ./data/output/model.tar.gz

In [ ]:
%%sh
# Specify an image name
image_name=tensorflow-training
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pluto-2"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

In [30]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin  049694889102.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [ ]:
!docker images

In [ ]:
!docker tag ciao:mondo 049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo

In [ ]:
!docker images

In [32]:
!docker push 049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo

The push refers to repository [049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training]

ddd4968f: Preparing 
0c667978: Preparing 
4cf9a903: Preparing 
22878ecf: Preparing 
db4f6696: Preparing 
f529ae49: Preparing 
75f13e59: Preparing 
0c6c24f9: Preparing 
a1e64371: Preparing 
29631b13: Preparing 
3f03efd0: Preparing 
9b75d5ee: Preparing 
370de786: Preparing 
a16f1a7a: Preparing 
fc8740d1: Preparing 
292691fc: Preparing 
6672bf9d: Preparing 
607be3da: Preparing 
afc9b360: Preparing 
f934e4df: Preparing 
80a2a6eb: Preparing 
a0538fa4: Preparing 
6d109c54: Preparing 
f0a068df: Preparing 
443b7673: Preparing 
37a36861: Preparing 
ecb4bd81: Preparing 
34b60c81: Preparing 
07d5d0ab: Preparing 
e6c8e003: Preparing 
ab822c1c: Preparing 
3609315f: Preparing 
ab822c1c: Pushed   755.5MB/731.1MBPushing  237.9MB/285.6MBPushing  41.33MB/72.81MBPushing  256.6MB/731.1MBPushing    561MB/731.1MBPushing  604.2MB/731.1MBmondo: digest: sha256:bb1fe01c372c2c0528f8ed3ed2ba55acde8433c2e6d14fc32ba375

In [35]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:mondo',
    role=role,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    hyperparameters=hyperparameters,
    output_path='s3://itsrizzoli-flaviomanna/data/output/'
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('s3://itsrizzoli-flaviomanna/data/input/cancer_data.csv')
print('##### ESTIMATOR FIT COMPLETED')

INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-11-05-15-26-20-914


##### ESTIMATOR FIT STARTED
2024-11-05 15:26:22 Starting - Starting the training job...
2024-11-05 15:26:46 Starting - Preparing the instances for training......
2024-11-05 15:27:55 Downloading - Downloading the training image..................
2024-11-05 15:30:33 Training - Training image download completed. Training in progress...2024-11-05 15:31:02.369960: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-05 15:31:02.374366: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-05 15:31:02.386614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730820662.407081      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already b

In [ ]:
!pip3 install tensorflow pandas numpy scikit-learn keras

In [ ]:
!pip3 freeze > requirements.txt

In [ ]:
!FLASK_APP=${PWD}/inference.py flask run --host 0.0.0.0 --port 8080

In [ ]:
curl http://localhost:8080/ping

In [17]:
%%sh
# Specify an image name
image_name=tensorflow-inference
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-pluto-2"
echo "fullname: ${fullname} ######################"

aws ecr describe-repositories --repository-names "${image_name}"
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}"
fi

image_name: tensorflow-inference ######################
account: 049694889102 ######################
region: us-east-1 ######################
fullname: 049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:2.12.0-cpu-py310-pluto-2 ######################



An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'tensorflow-inference' does not exist in the registry with id '049694889102'


{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:049694889102:repository/tensorflow-inference",
        "registryId": "049694889102",
        "repositoryName": "tensorflow-inference",
        "repositoryUri": "049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference",
        "createdAt": 1730816536.49,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

In [38]:
!docker build -t 049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo -f Dockerfile.inference .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (3/3)                                          docker:default
 => [internal] load build definition from Dockerfile.inference             0.0s
 => => transferring dockerfile: 311B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
[+] Building 0.2s (9/9) FINISHED                                 docker:default
 => [internal] load build definition from Dockerfile.inference             0.0s
 => => transferring dockerfile: 311B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

In [18]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 049694889102.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [39]:
!docker push 049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo

The push refers to repository [049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference]

10c05611: Preparing 
d9a7d2bc: Preparing 
37bf1d61: Preparing 
8df97c44: Preparing 
764747b4: Preparing 
1a68f579: Preparing 
2eef5eea: Preparing 
b5bcc575: Preparing 
5e5ce62f: Preparing 
757a305d: Preparing 
c7cf6f28: Preparing 
30092134: Preparing 
fb153852: Preparing 
0903db8c: Preparing 
19dec72a: Preparing 
df04f233: Preparing 
f2dbc490: Preparing 
ba0431f9: Preparing 
71536788: Preparing 
994107ae: Preparing 
3a4f83e7: Preparing 
d6748243: Preparing 
f2c1e372: Preparing 
cd2b5d6d: Preparing 
9a7d2bc: Pushed   2.641GB/2.621GBmondo: digest: sha256:2d13b30814161b50013ae2d59e7dadb8aa24e5b01cd71abaccdb47de7d5d487d size: 5558


In [28]:
!SM_MODEL_DIR=${PWD} FLASK_APP=${PWD}/inference.py flask run --host 0.0.0.0 --port 8080

2024-11-05 15:18:10.735528: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730819890.931234    5384 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730819891.011269    5384 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 15:18:11.782939: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
 * Serving Flask app '/home/ec2-user/SageMaker/2023-25.ML.UFS14/inference.py'
 * Debug mode: off
Address already in use
Port 

In [40]:
from sagemaker import get_execution_role
import boto3
import datetime
from time import gmtime, strftime

# https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html

my_session = boto3.session.Session()
aws_region = my_session.region_name

sagemaker_client = boto3.client('sagemaker', region_name=aws_region)

sagemaker_role = get_execution_role()

model_name = 'il-top'

# Create model
create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': '049694889102.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo',
        'ModelDataUrl': 's3://itsrizzoli-flaviomanna/data/output/tensorflow-training-2024-11-05-15-26-20-914/output/model.tar.gz',
    })

In [41]:

# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'la-mia-prima-api-inferenza-config'

instance_type = 'ml.p2.xlarge'

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name,
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = 'la-mia-prima-api-inferenza'

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)

Created EndpointConfig: arn:aws:sagemaker:us-east-1:049694889102:endpoint-config/la-mia-prima-api-inferenza-config
